In [2]:
import pathlib
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit

import json
import pickle as pkl

tqdm.pandas()

def display_f(df):
    with pd.option_context('display.max_colwidth', None), pd.option_context('display.max_rows', None), pd.option_context('display.max_columns', None):
        display(df)

## Possibly usefull interpolation tools :

def format_coefs(coefs):
    equation_list = [f"{coef}x^{i}" for i, coef in enumerate(coefs)]
    equation = "$" +  " + ".join(equation_list) + "$"
    replace_map = {"x^0": "", "x^1": "x", '+ -': '- '}
    for old, new in replace_map.items():
        equation = equation.replace(old, new)

    return equation

def model(df,fig, to_fit):
    y = df[[to_fit]].values
    X = df[['gdp']].values
    x_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
    for degree in [2]:
        poly = PolynomialFeatures(degree)
        poly.fit(X)
        X_poly = poly.transform(X)
        x_range_poly = poly.transform(x_range)
        model = Linearinterpolation(fit_intercept=False)
        model.fit(X_poly, df[to_fit])
        y_poly = model.predict(x_range_poly)
        equation = format_coefs(model.coef_.round(2))
        fig.add_traces(go.Scatter(x=x_range.squeeze(), y=y_poly, name=df.region.iloc[0]))
    return fig
    #model = model.fit(X, y)
    #x_range = np.linspace(X.min(), X.max(), 50)
    #y_range = model.predict(x_range.reshape(-1, 1))
   # print('x_range:',x_range) 
   # print('y_range:',y_range)
    #return x_range, y_range
    #return go.Scatter(x=x_range, y=y_range, name='interpolation Fit', fillcolor="red")

#def func_model(x, a, b, c, d):
#    return a * x + b * x*x + c*x*x*x + d

In [3]:
import dask.dataframe as dd

In [4]:
# Load the (flood frequency -> gdp share impacted) file
flood_gdp_share_f = pathlib.Path("../../../Data/Snakeruns/inputs/flood_country_gdp_share_full.json")
with flood_gdp_share_f.open('r') as f:
    flood_gdp_share = json.load(f)

# load the aggregated indicators database :    
#file = pathlib.Path("../../../Data/Floods_Run_1/interpolation_base.ods")
file2 = pathlib.Path("../../../Data/Floods_Run_1/New/general.csv")
prodloss = pathlib.Path("../../../Data/Floods_Run_1/New/prodloss.csv")
fdloss = pathlib.Path("../../../Data/Floods_Run_1/New/fdloss.csv")

df = pd.read_csv(file2)
df_prodloss = pd.read_csv(prodloss)
df_fdloss = pd.read_csv(fdloss)

#Load the flood databases
flood_base = pd.read_csv("./../../../Data/Flood_Base_clustered_all_EXIO3_regions_dmg_ge_0_0005.csv", keep_default_na=False)
flood_base

flood_base_full = pd.read_csv("./../../../Data/Flood_Base_clustered_all_EXIO3_regions_full.csv", keep_default_na=False)
flood_base_full


# Drop unused indicators that are heavy
df = df.drop(['top_5_sector_loss','rebuild_durations', '10_first_shortages'],axis=1)


    
df['prod_lost_aff'] = df['prod_lost_tot'] - df['prod_lost_unaff']
df['unaff_fd_unmet'] = df['tot_fd_unmet'] - df['aff_fd_unmet']
df = df.drop("Unnamed: 0", axis=1)
df[['type','flood_int']] = df['run_name'].str.extract(r"(?:[A-Z]{2})_type_(RoW|Full)_qdmg_((?:[\d_]+%)|(?:max))_Psi_(?:0_\d+)_inv_tau_(?:\d+)")

df_prodloss = df_prodloss.rename(columns={"Unnamed: 0":"run_name"})
df_prodloss

df_fdloss = df_fdloss.rename(columns={"Unnamed: 0":"run_name"})
df_fdloss

df_prodloss = df_prodloss.set_index("run_name")
df_fdloss = df_fdloss.set_index("run_name")
df = df.set_index('run_name')

display(df_prodloss)

,AT,AU,BE,BG,BR,CA,CH,CN,CY,CZ,...,SK,TR,TW,US,WA,WE,WF,WL,WM,ZA
run_name,,,,,,,,,,,,,,,,,,,,,
IN_type_Full_qdmg_10%_Psi_0_85_inv_tau_6,0.007513,0.014972,0.011521,0.000399,0.003668,0.008557,0.009369,0.179239,0.000000,0.002326,...,0.000000,0.003823,0.012370,0.260188,0.432903,0.002621,0.050282,0.048804,0.509555,0.001508
HU_type_Full_qdmg_1%_Psi_0_85_inv_tau_6,1.200137,0.133004,0.775580,0.082547,0.080024,0.093012,0.245833,2.742376,0.008245,0.504309,...,0.776802,0.179104,0.163841,0.837548,0.768668,0.849483,0.112182,0.166564,0.440345,0.038324
RU_type_Full_qdmg_80%_Psi_0_85_inv_tau_6,5.810530,4.877953,4.850114,2.939138,2.497591,3.119241,3.119156,86.120528,0.840433,7.417940,...,6.737868,8.479784,5.859596,21.381272,37.821311,27.789770,3.431085,12.530078,15.225634,1.537792
SI_type_Full_qdmg_80%_Psi_0_85_inv_tau_6,125.535925,4.417975,17.772951,5.226526,3.705543,4.838313,34.595124,143.429207,0.456156,26.436416,...,24.491675,20.446032,13.840455,39.067639,33.391288,101.484124,16.641598,8.564046,72.184378,1.727683
IN_type_Full_qdmg_99%_Psi_0_85_inv_tau_6,31.416586,40.093500,36.452154,4.077406,22.220300,29.152679,22.385003,641.426426,0.619988,10.001365,...,1.898718,17.698275,47.535369,533.140787,1582.487436,16.867400,207.478802,179.449384,2082.779005,14.886272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK_type_Full_qdmg_80%_Psi_0_85_inv_tau_6,159.525844,8.264697,69.495204,8.620714,5.664050,7.206544,38.670367,190.906313,2.276059,393.528094,...,3536.430443,21.347504,16.399457,63.604278,144.383893,120.067760,40.437115,13.514192,62.202475,3.856176
CN_type_Full_qdmg_75%_Psi_0_85_inv_tau_6,8.719305,7.520943,29.173900,1.607202,1.811239,7.965705,4.473570,60.703514,0.062425,2.491632,...,1.951040,1.642552,1.477385,33.475943,164.212866,22.853168,22.297045,20.307108,48.221299,1.269710
ID_type_Full_qdmg_10%_Psi_0_85_inv_tau_6,0.011014,0.100095,0.019219,0.001012,0.013296,0.021062,0.015581,0.878688,0.000000,0.004714,...,0.000581,0.008628,0.056927,0.291000,1.390329,0.006040,0.026574,0.028070,0.233527,0.007650


In [5]:
df_with_prodloss = df.join(df_prodloss)
df_with_fdloss = df.join(df_fdloss)
df_with_fdloss = df_with_fdloss.dropna(how="any",axis=0)
df_with_prodloss = df_with_prodloss.dropna(how="any",axis=0)
display_f(df_with_fdloss.columns)

Index(['region', 'gdp_dmg_share', 'tot_fd_unmet', 'aff_fd_unmet', 'shortage_b',
       'shortage_date_start', 'shortage_date_end', 'shortage_date_max',
       'shortage_ind_max', 'shortage_ind_mean', 'prod_gain_tot',
       'prod_lost_tot', 'prod_gain_unaff', 'prod_lost_unaff', 'inv_tau',
       'n_timesteps', 'has_crashed', 'prod_lost_aff', 'unaff_fd_unmet', 'type',
       'flood_int', 'AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CN', 'CY', 'CZ',
       'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'ID', 'IE',
       'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV', 'MT', 'MX', 'NL', 'NO', 'PL',
       'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR', 'TW', 'US', 'WA', 'WE', 'WF',
       'WL', 'WM', 'ZA'],
      dtype='object')

In [17]:
flood_base[flood_base.EXIO3_region=='LU']

,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,long,lat,date_cat,dmg_as_gdp_share
746890,r3_LU_d0cu,r3,LU,LU,2049-02-01,2049-02-01,522230577.0,11079.990,6.25,49.75,[2021-2050[,0.008221
746891,r3_LU_d1cu,r3,LU,LU,2071-02-21,2071-02-21,531929763.0,9599.246,6.25,49.75,[2071-2100[,0.008374
746892,r3_LU_d2cu,r3,LU,LU,2081-01-09,2081-01-09,504925943.0,8963.485,6.25,49.75,[2071-2100[,0.007949
746893,r3_LU_d3cu,r3,LU,LU,2109-07-03,2109-07-03,609774986.0,16115.000,6.25,49.75,,0.009599


In [10]:
from scipy.interpolate import interp1d
# linear interpolation per group
   

# apply linear regresion using numpy
def linReg(x, y):
    '''linear interpolation using numpy starting from two one dimensional numpy arrays'''
    A = np.vstack([x, np.ones(len(x))]).T
    model, resid = np.linalg.lstsq(A, y, rcond=None)[:2]
    r2 = 1 - resid / (y.size * y.var())
    return pd.Series({'slope':model[0], 'intercept': model[1], 'R-square':r2[0]})

def interpolations_coefs(reg_df,x_values,ys):
    dico = {}
    for y in ys:
        #reg_coefs = df.groupby('region').apply(lambda x: linReg(x[x_values], x[y]))
        reg_coefs = reg_df.groupby('region').apply(lambda x: interp1d(x[x_values], x[y],fill_value="extrapolate"))
        dico[y] = reg_coefs
    return pd.concat(dico.values(), axis=1, keys=dico.keys())

fdloss_interpolations_coefs_df = interpolations_coefs(df_with_fdloss,"gdp_dmg_share",['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA'])

prodloss_interpolations_coefs_df = interpolations_coefs(df_with_prodloss,"gdp_dmg_share",['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA'])

In [11]:
#print(prodloss_interpolations_coefs_df.columns)
#print(len(prodloss_interpolations_coefs_df.index))
test = prodloss_interpolations_coefs_df.rename_axis("flooded_region").melt(ignore_index=False, var_name="region", value_name="interp")

In [12]:
test

,region,interp
flooded_region,,
AT,AT,<scipy.interpolate.interpolate.interp1d object...
AU,AT,<scipy.interpolate.interpolate.interp1d object...
BE,AT,<scipy.interpolate.interpolate.interp1d object...
BG,AT,<scipy.interpolate.interpolate.interp1d object...
BR,AT,<scipy.interpolate.interpolate.interp1d object...
...,...,...
WE,ZA,<scipy.interpolate.interpolate.interp1d object...
WF,ZA,<scipy.interpolate.interpolate.interp1d object...
WL,ZA,<scipy.interpolate.interpolate.interp1d object...


In [25]:
test2 = flood_base[["final_cluster","EXIO3_region","dmg_as_gdp_share"]].copy()
test2[['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']] = 0
len(test2.EXIO3_region.unique())
test2 = test2.set_index("EXIO3_region").sort_index()

45

In [28]:
test3 = dd.from_pandas(test2,npartitions=45)

In [29]:
test3.divisions

('AT', 'CA', 'CH', 'CZ', 'IT', 'NL', 'WE', 'WL', 'WM', 'ZA')

In [21]:
test2.melt(id_vars=["final_cluster","EXIO3_region", "dmg_as_gdp_share"], var_name="indirect_region")

,final_cluster,EXIO3_region,dmg_as_gdp_share,indirect_region,value
0,r1_AF_d0c0,WA,6.478794e-09,AT,0
1,r1_AF_d0c1,WA,9.358541e-08,AT,0
2,r1_AF_d0c10,WA,1.204101e-07,AT,0
3,r1_AF_d0c11,WA,7.592000e-08,AT,0
4,r1_AF_d0c2,WA,3.125526e-10,AT,0
...,...,...,...,...,...
100535451,r7_ZW_d99c8,WF,5.581549e-09,ZA,0
100535452,r7_ZW_d99c9,WF,3.621383e-09,ZA,0
100535453,r7_ZW_d9c0,WF,5.253989e-09,ZA,0
100535454,r7_ZW_d9c1,WF,1.865750e-08,ZA,0


In [ ]:
prodlossdict = prodloss_interpolations_coefs_df.to_dict()
fdlossdict = fdloss_interpolations_coefs_df.to_dict()


In [68]:
def projected_prodloss(impacted,dmg):
    return lambda region: prodlossdict[region][impacted](dmg)

def projected_prodloss_row(row,regions):
    return [projected_prodloss(impacted=row['EXIO3_region'],dmg=row['dmg_as_gdp_share'])(region) for region in regions]

def projected_fdloss(impacted,dmg):
    return lambda region: fdlossdict[region][impacted](dmg)

def projected_fdloss_row(row):
    return projected_fdloss(impacted=row['EXIO3_region'],dmg=row['dmg_as_gdp_share'])

def projected_indicators(row, ind):
    row.apply
    return pd.Series(dico)

def prodloss_indic(row):
    return projected_indicators(row, interpolations_coefs_df=prodloss_interpolations_coefs_df)
    
def fdloss_indic(row):
    return projected_indicators(row, interpolations_coefs_df=fdloss_interpolations_coefs_df)

In [84]:
regions = ['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']

test = flood_base.head().copy()
test

,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,long,lat,date_cat,dmg_as_gdp_share
0,r1_AF_d0c0,r1,WA,AF,2009-08-03,2009-08-03,22353.800,8322.58,70.75,38.25,[2006-2020[,6.478794e-09
1,r1_AF_d0c1,r1,WA,AF,2009-08-04,2009-08-04,322898.000,6862.00,70.25,37.75,[2006-2020[,9.358541e-08
2,r1_AF_d0c10,r1,WA,AF,2009-08-04,2009-08-04,415451.200,101934.18,71.00,34.75,[2006-2020[,1.204101e-07
3,r1_AF_d0c11,r1,WA,AF,2009-08-04,2009-08-04,261947.000,40772.82,70.75,34.25,[2006-2020[,7.592000e-08
4,r1_AF_d0c2,r1,WA,AF,2009-08-03,2009-08-03,1078.401,0.00,72.75,37.75,[2006-2020[,3.125526e-10


In [ ]:
flood_base[regions] = flood_base.progress_apply(lambda row : projected_prodloss_row(row, regions), axis=1, result_type='expand')

In [86]:
test

,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,long,lat,...,SK,TR,TW,US,WA,WE,WF,WL,WM,ZA
0,r1_AF_d0c0,r1,WA,AF,2009-08-03,2009-08-03,22353.800,8322.58,70.75,38.25,...,-2.462085088514582e-05,0.002381480684286858,0.004525931080202302,0.027807859141546883,0.23815721711901516,0.0012247323041084108,0.002781044598735503,0.00537106563383117,0.02069342065259338,0.0012760616175215918
1,r1_AF_d0c1,r1,WA,AF,2009-08-04,2009-08-04,322898.000,6862.00,70.25,37.75,...,0.00023820534709675462,0.03292153256503441,0.04095482708007088,0.19407893559088737,0.21358329610614715,0.004890933828395192,0.009728965431759303,0.022972114888072433,0.06658714781622974,0.002583581407695661
2,r1_AF_d0c10,r1,WA,AF,2009-08-04,2009-08-04,415451.200,101934.18,71.00,34.75,...,0.0005289721853063025,0.04203677211084332,0.05184556620040803,0.2274457095145684,0.21598373914226135,0.006960770864653837,0.012690783584730008,0.028586484589845883,0.081176659292124,0.004236274025223397
3,r1_AF_d0c11,r1,WA,AF,2009-08-04,2009-08-04,261947.000,40772.82,70.75,34.25,...,4.672056260837247e-05,0.026918682684570008,0.033782720225595086,0.17210521412666702,0.21200248200685942,0.0035278406322982385,0.007778457202527712,0.019274766252986957,0.05697921082466995,0.0014951990748218872
4,r1_AF_d0c2,r1,WA,AF,2009-08-03,2009-08-03,1078.401,0.00,72.75,37.75,...,-3.0410748757146653e-05,0.0002018736303721451,0.0019271354203547301,0.01494820647293943,0.24050559969589871,0.0010226648683755817,0.002339420138548639,0.0041369489024141925,0.017472500466399313,0.0012598499034799896


In [43]:
list(prodlossdict['AT'].keys())

['AT',
 'AU',
 'BE',
 'BG',
 'BR',
 'CA',
 'CH',
 'CN',
 'CZ',
 'DE',
 'EE',
 'ES',
 'FI',
 'FR',
 'GB',
 'GR',
 'HR',
 'HU',
 'ID',
 'IE',
 'IN',
 'IT',
 'JP',
 'KR',
 'LT',
 'LV',
 'MX',
 'NO',
 'PL',
 'PT',
 'RO',
 'RU',
 'SE',
 'SI',
 'SK',
 'TR',
 'US',
 'ZA']

In [ ]:
result_prodloss = flood_base.progress_apply(prodloss_indic,axis=1)

In [ ]:
result_fdloss = flood_base.progress_apply(fdloss_indic,axis=1)

In [ ]:
final_res_prodloss = flood_base.join(result_prodloss.dropna(how="all",axis=0))
final_res_prodloss

final_res_fdloss = flood_base.join(result_fdloss.dropna(how="all",axis=0))
final_res_fdloss

In [68]:
final_res_prodloss = final_res_prodloss.drop("date_cat", axis=1)
final_res_fdloss = final_res_fdloss.drop("date_cat",axis=1)

final_res_prodloss = final_res_prodloss.dropna(how="any", axis=0)
final_res_fdloss = final_res_fdloss.dropna(how="any", axis=0)
final_res_fdloss

,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,coord,dmg_as_gdp_share,...,SK,TR,TW,US,WA,WE,WF,WL,WM,ZA
23,r1_AT_d0cu,r1,AT,AT,2085-05-13,2085-05-13,2.933136e+08,22371.230,"(0.8333947178272924, 0.23125612588924865)",0.000738,...,2.747145e+06,1.658675e+06,1.299886e+06,1.970160e+07,5.817289e+06,3.809175e+06,2.466334e+06,2.011022e+06,4.893292e+06,5.380176e+05
24,r1_AT_d1cu,r1,AT,AT,2063-06-23,2063-06-23,2.186195e+08,79415.390,"(0.8246680715673207, 0.17889624832941878)",0.000550,...,1.985341e+06,1.204888e+06,9.424425e+05,1.422089e+07,4.218710e+06,2.779137e+06,1.796618e+06,1.457330e+06,3.549911e+06,3.908750e+05
25,r1_AT_d2c0,r1,AT,AT,2078-05-23,2078-05-23,2.284664e+09,87959.090,"(0.8246680715673207, 0.2748893571891069)",0.005746,...,2.233370e+07,1.343995e+07,1.014100e+07,1.472398e+08,4.611732e+07,2.992861e+07,2.013131e+07,1.613214e+07,3.829556e+07,4.578957e+06
26,r1_AT_d2c1,r1,AT,AT,2078-05-22,2078-05-24,1.060480e+09,74091.102,"(0.815941425307349, 0.24870941840919197)",0.002667,...,1.038859e+07,6.247214e+06,4.758407e+06,6.986753e+07,2.157364e+07,1.399149e+07,9.337150e+06,7.529047e+06,1.794823e+07,2.105213e+06
27,r1_AT_d3c0,r1,AT,AT,2078-05-10,2078-05-11,3.946924e+08,43443.988,"(0.815941425307349, 0.25743606466916363)",0.000993,...,3.873436e+06,2.323751e+06,1.821330e+06,2.761577e+07,8.178018e+06,5.298318e+06,3.450597e+06,2.834567e+06,6.844725e+06,7.567400e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,r7_SK_d2cu,r7,SK,SK,2047-05-21,2047-05-21,2.968003e+08,40925.730,"(0.8508480103472357, 0.3272492347489368)",0.003157,...,1.320838e+08,1.952901e+06,1.370179e+06,1.473716e+07,7.113907e+06,5.193376e+06,2.793023e+06,1.891323e+06,5.063353e+06,4.210050e+05
24067,r7_US_d120cu,r7,US,US,2022-02-21,2022-02-21,1.285912e+10,834106.000,"(0.5890486225480862, -2.0463985479633515)",0.000672,...,2.436708e+07,1.771144e+08,3.516790e+08,2.283222e+11,1.703253e+09,9.471037e+07,3.092663e+08,9.982192e+08,9.420295e+08,7.787236e+07
24068,r7_US_d269c0,r7,US,US,2062-03-13,2062-03-13,1.285912e+10,834106.000,"(0.5890486225480862, -2.0463985479633515)",0.000672,...,2.436708e+07,1.771144e+08,3.516790e+08,2.283222e+11,1.703253e+09,9.471037e+07,3.092663e+08,9.982192e+08,9.420295e+08,7.787236e+07
24069,r7_US_d95c8,r7,US,US,2035-03-01,2035-03-01,1.285912e+10,834106.000,"(0.5890486225480862, -2.0463985479633515)",0.000672,...,2.436708e+07,1.771144e+08,3.516790e+08,2.283222e+11,1.703253e+09,9.471037e+07,3.092663e+08,9.982192e+08,9.420295e+08,7.787236e+07


In [ ]:
from datetime import date

final_res_prodloss['year'] = pd.DatetimeIndex(final_res_prodloss['date_start']).year
final_res_prodloss['date_start'] = pd.DatetimeIndex(final_res_prodloss['date_start'])
bins = [np.datetime64(date(1970,12,31)), np.datetime64(date(2005,12,31)), np.datetime64(date(2016,12,31)), np.datetime64(date(2035,12,31)), np.datetime64(date(2050,12,31)), np.datetime64(date(2070,12,31)), np.datetime64(date(2100,12,31)), np.datetime64(date(2150,12,31))] #type: ignore
group_names = ['historic','[2006-2016[', '[2016-2035[', '[2035-2050[', '[2051-2070[', '[2071-2100[', '2100+']
final_res_prodloss['date_cat'] = pd.cut(final_res_prodloss.date_start, bins, labels=group_names)
final_res_prodloss

final_res_fdloss['year'] = pd.DatetimeIndex(final_res_fdloss['date_start']).year
final_res_fdloss['date_start'] = pd.DatetimeIndex(final_res_fdloss['date_start'])
bins = [np.datetime64(date(1970,12,31)), np.datetime64(date(2005,12,31)), np.datetime64(date(2016,12,31)), np.datetime64(date(2035,12,31)), np.datetime64(date(2050,12,31)), np.datetime64(date(2070,12,31)), np.datetime64(date(2100,12,31)), np.datetime64(date(2150,12,31))] #type: ignore
group_names = ['historic','[2006-2016[', '[2016-2035[', '[2035-2050[', '[2051-2070[', '[2071-2100[', '2100+']
final_res_fdloss['date_cat'] = pd.cut(final_res_fdloss.date_start, bins, labels=group_names)
final_res_fdloss

In [ ]:
# integrate over date_cat (to get indicator sum over the period)
# for both total and by impacted region (to differentiate imported and local indirect damages)

projected_prodloss_total = final_res_prodloss.groupby(['model', 'date_cat'])[['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']].agg("sum")

projected_prodloss_by_region_impacted = final_res_prodloss.groupby(['model', 'EXIO3_region', 'date_cat'])[['total_dmg' ,'AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']].agg("sum")

projected_fdloss_total = final_res_fdloss.groupby(['model', 'date_cat'])[['AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']].agg("sum")

projected_fdloss_by_region_impacted = final_res_fdloss.groupby(['model', 'EXIO3_region', 'date_cat'])[['total_dmg' ,'AT', 'AU', 'BE', 'BG', 'BR',
       'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB',
       'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV',
       'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR',
       'TW', 'US', 'WA', 'WE', 'WF', 'WL', 'WM', 'ZA']].agg("sum")


In [71]:
def get_impacted_loss(row):
    region = row["EXIO3_region"]
    return row[region]

projected_prodloss_by_region_impacted = projected_prodloss_by_region_impacted.reset_index()
local_prodloss = projected_prodloss_by_region_impacted[["model", "EXIO3_region","date_cat"]].copy()
local_prodloss['projected_prodloss'] = projected_prodloss_by_region_impacted.groupby(["model", "EXIO3_region","date_cat"]).apply(get_impacted_loss).sum(axis=1)
local_prodloss = local_prodloss.set_index(["model", "date_cat", "EXIO3_region"])
local_prodloss

projected_fdloss_by_region_impacted = projected_fdloss_by_region_impacted.reset_index()
local_fdloss = projected_fdloss_by_region_impacted[["model", "EXIO3_region","date_cat"]].copy()
local_fdloss['projected_fdloss'] = projected_fdloss_by_region_impacted.groupby(["model", "EXIO3_region","date_cat"]).apply(get_impacted_loss).sum(axis=1)
local_fdloss = local_fdloss.set_index(["model", "date_cat", "EXIO3_region"])
local_fdloss

expected_impacted_prod_loss
model date_cat    EXIO3_region                             
r1    historic    AT                           0.000000e+00
      [2006-2016[ AT                           0.000000e+00
      [2016-2035[ AT                           0.000000e+00
      [2035-2050[ AT                           0.000000e+00
      [2051-2070[ AT                           4.131497e+08
...                                                     ...
r7    [2016-2035[ ZA                           7.887097e+08
      [2035-2050[ ZA                           0.000000e+00
      [2051-2070[ ZA                           0.000000e+00
      [2071-2100[ ZA                           0.000000e+00
      2100+       ZA                           0.000000e+00

[1862 rows x 1 columns]

In [72]:
projected_prodloss_total = projected_prodloss_total.melt(value_name="projected_prodloss",var_name="region", ignore_index=False)

In [73]:
projected_prodloss_total['EXIO3_region'] = projected_prodloss_total['region'].map({v:v[-2:] for v in np.unique(projected_prodloss_total['region'])})
projected_prodloss_total = projected_prodloss_total.drop('region',axis=1)
projected_prodloss_total = projected_prodloss_total.reset_index().set_index(["model","date_cat","EXIO3_region"])
projected_prodloss_total

prodloss
model date_cat    EXIO3_region              
r1    historic    AT            0.000000e+00
      [2006-2016[ AT            9.258535e+09
      [2016-2035[ AT            3.789989e+10
      [2035-2050[ AT            1.812537e+10
      [2051-2070[ AT            5.881780e+10
...                                      ...
r7    [2016-2035[ ZA            5.550619e+09
      [2035-2050[ ZA            2.394626e+09
      [2051-2070[ ZA            2.486554e+09
      [2071-2100[ ZA            1.030885e+09
      2100+       ZA            0.000000e+00

[2401 rows x 1 columns]

In [74]:
projected_fdloss_total = projected_fdloss_total.melt(value_name="projected_fdloss",var_name="region", ignore_index=False)
projected_fdloss_total['EXIO3_region'] = projected_fdloss_total['region'].map({v:v[-2:] for v in np.unique(projected_fdloss_total['region'])})
projected_fdloss_total = projected_fdloss_total.drop('region',axis=1)
projected_fdloss_total = projected_fdloss_total.reset_index().set_index(["model","date_cat","EXIO3_region"])
projected_fdloss_total

fdloss
model date_cat    EXIO3_region              
r1    historic    AT            0.000000e+00
      [2006-2016[ AT            3.470103e+09
      [2016-2035[ AT            1.385864e+10
      [2035-2050[ AT            7.068857e+09
      [2051-2070[ AT            1.856464e+10
...                                      ...
r7    [2016-2035[ ZA            6.484649e+09
      [2035-2050[ ZA            5.764682e+09
      [2051-2070[ ZA            1.420685e+10
      [2071-2100[ ZA            3.359884e+10
      2100+       ZA            0.000000e+00

[2401 rows x 1 columns]

In [75]:
projected_prodloss_total['direct_damage'] = final_res_prodloss.groupby(['model', 'date_cat'])['total_dmg'].sum()
projected_prodloss_total = projected_prodloss_total.reset_index()

In [76]:
projected_fdloss_total

fdloss
model date_cat    EXIO3_region              
r1    historic    AT            0.000000e+00
      [2006-2016[ AT            3.470103e+09
      [2016-2035[ AT            1.385864e+10
      [2035-2050[ AT            7.068857e+09
      [2051-2070[ AT            1.856464e+10
...                                      ...
r7    [2016-2035[ ZA            6.484649e+09
      [2035-2050[ ZA            5.764682e+09
      [2051-2070[ ZA            1.420685e+10
      [2071-2100[ ZA            3.359884e+10
      2100+       ZA            0.000000e+00

[2401 rows x 1 columns]

In [ ]:
df_box = projected_prodloss_total
fig = px.box(df_box,
             facet_row="EXIO3_region",
             x = "date_cat",
             y="projected_prodloss",
             facet_row_spacing= 0.002,
             height=15000,
             hover_data=['model']
      )
fig.update_yaxes(matches=None)
fig.update_xaxes(matches=None)

In [89]:
imported_prodloss = projected_prodloss_total['prodloss'] - local_prodloss['projected_prodloss']
imported_prodloss

imported_fdloss = projected_fdloss_total['prodloss'] - local_prodloss['projected_fdloss']
imported_fdloss

model  date_cat  EXIO3_region
r1     2100+     AT              1.309288e+11
                 AU             -3.960600e+11
                 BE              1.982141e+11
                 BG              1.380297e+10
                 BR             -2.813829e+11
                                     ...     
r7     historic  WE                       NaN
                 WF                       NaN
                 WL                       NaN
                 WM                       NaN
                 ZA              0.000000e+00
Length: 2401, dtype: float64

In [78]:
projected_fdloss_total = projected_fdloss_total.reset_index()
projected_fdloss_total

,model,date_cat,EXIO3_region,fdloss
0,r1,historic,AT,0.000000e+00
1,r1,[2006-2016[,AT,3.470103e+09
2,r1,[2016-2035[,AT,1.385864e+10
3,r1,[2035-2050[,AT,7.068857e+09
4,r1,[2051-2070[,AT,1.856464e+10
...,...,...,...,...
2396,r7,[2016-2035[,ZA,6.484649e+09
2397,r7,[2035-2050[,ZA,5.764682e+09
2398,r7,[2051-2070[,ZA,1.420685e+10
2399,r7,[2071-2100[,ZA,3.359884e+10


In [ ]:
df_box = projected_fdloss_total
fig = px.box(df_box,
             facet_row="EXIO3_region",
             x = "date_cat",
             y="projected_fdloss",
             facet_row_spacing= 0.002,
             height=15000,
             hover_data=['model']
      )
fig.update_yaxes(matches=None)
fig.update_xaxes(matches=None)

In [9]:
import plotly.express as px
import geopandas as geopd
import pickle as pkl
import country_converter as coco
cc = coco.CountryConverter()

w_regions =  {'WA':'RoW Asia and Pacific','WE':'RoW Europe', 'WF':'RoW Africa', 'WL':'RoW America', 'WM':'RoW Middle East'}

world = geopd.read_file(geopd.datasets.get_path('naturalearth_lowres'))
# fix for bug https://github.com/geopandas/geopandas/issues/1041
world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
world.loc[world['name'] == 'N. Cyprus', 'iso_a3'] = 'CYP'
world.loc[world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'RKS'

dico2 = cc.get_correspondence_dict('ISO3','EXIO3')
dico2 = {k:v[0] for k,v in dico2.items()}
world = world.reset_index()
world['EXIO3_region'] = world['iso_a3'].map(dico2)
world = world.set_index('EXIO3_region')
world

,index,pop_est,continent,name,iso_a3,gdp_md_est,geometry
EXIO3_region,,,,,,,
WA,0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
WF,1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
WF,2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
CA,3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
US,4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...,...
WE,172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
WE,173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
NaN,174,1895250,Europe,Kosovo,RKS,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."


In [ ]:
with pathlib.Path("../../../Data/EXIO_world.geojson").open('r') as f:
    geojs = json.load(f)
#df.loc[df['name'] == 'France', ''] = 0

fig = go.Figure()
fig = fig.add_trace(go.Choropleth(
    geojson=geojs,
    locations=df.index,
    z = df['test'],
    featureidkey='properties.DESIRE code',
    colorscale = [[0,'rgb(200, 200, 200)']],
    showlegend=False,
    coloraxis=None,
    colorbar=None,
))
fig.update_traces(showscale=False)

scale=df['projected_tot_fd_unmet'].max()

fig = fig.add_trace(go.Scattergeo(
    geojson=geojs,
    locations=df.index,
    marker = dict(
        color=df['projected_tot_fd_unmet'],
        size = 8,
        opacity = 0.7,
        showscale=False
                 ),
        
    featureidkey='properties.DESIRE code'
))
go.scattergeo.Marker()
fig.update_layout(
    autosize=False,
    width=1600,
    height=900,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [91]:
projected_prodloss_total = projected_prodloss_total.reset_index()
projected_prodloss_total = projected_prodloss_total.set_index(['model','date_cat','EXIO3_region'])

In [92]:
df = projected_prodloss_total.loc['r4']
df

,,prodloss,direct_damage
date_cat,EXIO3_region,,
historic,AT,0.000000e+00,0.000000e+00
[2006-2016[,AT,5.455028e+09,3.125992e+11
[2016-2035[,AT,3.795284e+10,5.786674e+11
[2035-2050[,AT,3.038728e+10,7.086573e+11
[2051-2070[,AT,3.032384e+10,1.403461e+12
...,...,...,...
[2016-2035[,ZA,4.146130e+09,5.786674e+11
[2035-2050[,ZA,2.338721e+09,7.086573e+11
[2051-2070[,ZA,-1.898103e+11,1.403461e+12


In [94]:
geodf = world.join(df.reset_index().set_index("EXIO3_region"))
#geodf = geodf.rename(columns={0:"prodloss"})
#geodf = geodf.dropna(how='any',axis=0)
geodf

,pop_est,continent,name,gdp_md_est,geometry,date_cat,prodloss,direct_damage
AFG,34124811,Asia,Afghanistan,64080.0,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",NaN,NaN,NaN
AGO,29310273,Africa,Angola,189000.0,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4...",NaN,NaN,NaN
ALB,3047987,Europe,Albania,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000...",NaN,NaN,NaN
ARE,6072475,Asia,United Arab Emirates,667200.0,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",NaN,NaN,NaN
ARG,44293293,South America,Argentina,879400.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
VUT,282814,Oceania,Vanuatu,723.0,"MULTIPOLYGON (((167.21680 -15.89185, 167.84488...",NaN,NaN,NaN
YEM,28036829,Asia,Yemen,73450.0,"POLYGON ((52.00001 19.00000, 52.78218 17.34974...",NaN,NaN,NaN
ZAF,54841552,Africa,South Africa,739100.0,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",NaN,NaN,NaN
ZMB,15972000,Africa,Zambia,65170.0,"POLYGON ((30.74001 -8.34001, 31.15775 -8.59458...",NaN,NaN,NaN


In [164]:
geodf['sign'] = geodf['prodloss'].gt(0)
geodf['sign'] = geodf['sign'].map({True:"prodloss",False:"prodgain"})
geodf

,iso_a3,pop_est,continent,name,gdp_md_est,geometry,date_cat,prodloss,sign
EXIO3_region,,,,,,,,,
AT,AUT,8754413,Europe,Austria,416600.0,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",historic,0.000000e+00,prodgain
AT,AUT,8754413,Europe,Austria,416600.0,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",[2006-2016[,5.455028e+09,prodloss
AT,AUT,8754413,Europe,Austria,416600.0,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",[2016-2035[,3.795284e+10,prodloss
AT,AUT,8754413,Europe,Austria,416600.0,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",[2035-2050[,3.038728e+10,prodloss
AT,AUT,8754413,Europe,Austria,416600.0,"POLYGON ((16.97967 48.12350, 16.90375 47.71487...",[2051-2070[,3.032384e+10,prodloss
...,...,...,...,...,...,...,...,...,...
ZA,ZAF,54841552,Africa,South Africa,739100.0,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",[2016-2035[,4.146130e+09,prodloss
ZA,ZAF,54841552,Africa,South Africa,739100.0,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",[2035-2050[,2.338721e+09,prodloss
ZA,ZAF,54841552,Africa,South Africa,739100.0,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",[2051-2070[,-1.898103e+11,prodgain


In [21]:
flood_base = pd.read_csv("./../../../Data/Flood_Base_clustered_all_EXIO3_regions_full.csv", keep_default_na=False)
flood_base = flood_base[flood_base.total_dmg>0].copy()
flood_base = flood_base[flood_base.model=='r3'].copy()
flood_base

,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,long,lat
1246283,r3_AF_d1000c0,r3,WA,AF,2116-03-10,2116-03-11,3.895817e+04,58549.04000,69.00,35.75
1246284,r3_AF_d1000c1,r3,WA,AF,2116-03-11,2116-03-11,1.619335e+05,536.50410,68.75,36.25
1246285,r3_AF_d1000c2,r3,WA,AF,2116-03-10,2116-03-11,1.252050e+05,816.15886,69.75,36.75
1246289,r3_AF_d1001c1,r3,WA,AF,2116-03-21,2116-03-21,1.812905e+05,600.63610,68.75,36.25
1246290,r3_AF_d1001c2,r3,WA,AF,2116-03-21,2116-03-21,2.573824e+05,863.31510,68.75,36.75
...,...,...,...,...,...,...,...,...,...,...
2202496,r3_ZW_d9c36,r3,WF,ZW,2015-01-10,2015-01-10,2.608556e+03,1647.98000,29.25,-17.25
2202497,r3_ZW_d9c37,r3,WF,ZW,2015-01-10,2015-01-10,5.286347e+03,1739.58400,29.25,-17.75
2202499,r3_ZW_d9c4,r3,WF,ZW,2015-01-09,2015-01-09,4.538183e+03,5982.84500,30.25,-16.75
2202500,r3_ZW_d9c5,r3,WF,ZW,2015-01-09,2015-01-09,8.494293e+07,4489.00000,30.25,-17.25


,final_cluster,model,EXIO3_region,country,date_start,date_end,total_dmg,total_pop,long,lat,ISO3,date_cat,long_jitted,lat_jitted


In [ ]:
### PLOT DIRECT DAMAGES FROM FLOODS

from datetime import date
import country_converter as coco
import ast


#flood_base_full = pd.read_csv("./../../../Data/Flood_Base_clustered_all_EXIO3_regions_full.csv", keep_default_na=False)

cc = coco.CountryConverter()
flood_base["ISO3"] = flood_base["country"].map({k : cc.convert(k, to="ISO3") for k in flood_base.country.unique() })
flood_base["country"] = flood_base["country"].map({k : cc.convert(k, to="name_short") for k in flood_base.country.unique() })

flood_base.date_start = pd.to_datetime(flood_base.date_start)
bins = [np.datetime64(date(1970,12,31)), np.datetime64(date(2005,12,31)), np.datetime64(date(2016,12,31)), np.datetime64(date(2035,12,31)), np.datetime64(date(2050,12,31)), np.datetime64(date(2070,12,31)), np.datetime64(date(2100,12,31)), np.datetime64(date(2150,12,31))] #type: ignore
group_names = ['historic','[2006-2016[', '[2016-2035[', '[2035-2050[', '[2051-2070[', '[2071-2100[', '[2100-++[']
flood_base['date_cat'] = pd.cut(flood_base.date_start, bins, labels=group_names)

sigma = 0.04
flood_base['long_jitted'] = flood_base['long'].apply(lambda x: np.random.normal(x, sigma))
flood_base['lat_jitted'] = flood_base['lat'].apply(lambda x: np.random.normal(x, sigma))

px.defaults.template = "seaborn"

floods_in_world = flood_base.set_index('ISO3').join(world)
floods_in_world = floods_in_world.sort_values(by='date_cat')

floods_in_world = floods_in_world.rename(columns={"model":"Model", "dmg_as_gdp_share":"Direct damage over GDP", "country":"Country", "date_cat":"Period"})

fig = px.scatter_geo(floods_in_world, 
                     lon="long_jitted", lat="lat_jitted",
                     animation_frame="Period",
                     color="Model",
                     #size="Direct damage over GDP",
                     color_discrete_sequence=px.colors.qualitative.Dark2,
                     opacity=0.8,
                     size_max=20,
                     height=850,
                     hover_data=['Country'],
                    )

fig.update_traces(marker=dict(line=dict(width=1,color='DarkSlateGrey')))

fig.update_layout(
    title = "Global projection of river floods and their intensity",
    margin={"r":10, "t":40,"l":10,"b":10},
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

fig.update_geos(
    lataxis_range=[-55,80], lonaxis_range=[-150, 180],
    #showcountries=True,
    #showland=True, landcolor="LightGreen",
    #showocean=True, oceancolor="CornflowerBlue",
    showlakes=True, lakecolor="DodgerBlue",
    showrivers=True, rivercolor="DodgerBlue",
    resolution = 50
)

fig.show()
fig.write_html("./../../../Data/Floods_Run_1/Flood_map.html")